# Geospatial example using ArcPy: WDPA change statistics

## Background

After countries submit spatial data to UNEP-WCMC, we incorporate them in World Database on Protected Areas (WDPA), which is released to the public on a monthly basis. It is imperative that quality control is in place to ensure:

1. There are no inherent quality issues in the newly submitted data
2. Update is correctly incorporated as intended (it could introduce serious errors too, if one is not careful!)

Furthermore, it is often useful for data curators to keep a record of what happened during an update, in case any retrospective examinations require this information in the future.

## What needs to be done

For any two given WDPA releases with the same schema, a pair-wise comparison is calculated for each of the required fields, for example: `name`, `orig_name`, `desig` and etc. If there is a difference, record both the old and new values. It is also necessary to flag records that have been added or deleted.

Geometric changes are a must as they, in theory, indicate a change in protected areas boundary. However, it is also possible that the difference may reflect the fact that a better boundary (or mapped at a larger scale) is provided. Thus the check should provide an indication of these differences but defer the judgement to programme staff.

## Challenges

1. Must be automated as it is not feasible to undertake examination each time when an update is ready for release
1. Differences must be identified at the field level
2. The large size of WDPA datasets

## Solution

A production ready solution can be found [here](https://github.com/Yichuans/geoprocessing/blob/master/snippet/check_wdpa_update.py)

## 1. Explore

Fine grain information retrieval using `arcpy.da.SearchCursor`

In [107]:
def get_row_info(wdpa_pid, data, fields_list):
    """"
    depending on the datatype of pid, generate correct SQL
    <wdpa_pid>, <fc/fl>, <list of fields>"""
    where_name = (isinstance(wdpa_pid, int) or isinstance(wdpa_pid, float)) and str(wdpa_pid) or ('\'' + str(wdpa_pid) + '\'')
    
    where_clause = "wdpa_pid = " + where_name

    with arcpy.da.SearchCursor(data, fields_list, where_clause=where_clause) as cur:
        for each in cur:
            # if found return the first value
            return each
        
    # if not found return None
    return None

In [108]:
a = get_row_info(wdpa_pid='4342', fields_list=['SHAPE@'], data=old_data)
b = get_row_info(wdpa_pid='4342', fields_list=['SHAPE@'], data=new_data)

In [109]:
a[0]!=b[0]

False

In [110]:
type(a[0])

arcpy.arcobjects.geometries.Polygon

In [111]:
isinstance(a[0], arcpy.Geometry)

True

In [112]:
geom = a[0]

Area in square degree

In [113]:
geom.area?

In [114]:
geom.area

3.3789838525636344e-06

Geodesic Area

In [115]:
geom.getArea()

28585.385716621553

In [116]:
geom.getArea(units='SQUAREKILOMETERS')

0.02858538571662155

Project to Mollweide

In [117]:
geom.projectAs(arcpy.SpatialReference(54009)).getArea()

28585.38571649975

In [118]:
geom.partCount

1

In [119]:
geom.centroid.X

15.039111922014502

In [120]:
b[0].centroid.X

15.039111922247134

In [121]:
geom.pointCount

46

In [122]:
b[0].pointCount

46

In [123]:
from collections import deque
queue = deque(["Eric", "John", "Michael"])
queue.append("Terry")           # Terry arrives
queue.append("Graham")          # Graham arrives
queue.popleft()                 # The first to arrive now leaves

'Eric'

In [124]:
%%timeit
q2 = deque(range(1000))
while q2:
    q2.popleft()

63.8 µs ± 269 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [125]:
%%timeit
q3 = deque(range(1000))
while q3:
     q3.pop()

64.1 µs ± 980 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 2. Refactor and encapsulate

In [126]:
import numpy as np
import pandas as pd
import arcpy

Create a convenient function that return `row` from an input feature class, given `id` and `fields_list`

In [127]:
def get_row_info(wdpa_pid, data, fields_list):
    """"
    depending on the datatype of pid, generate correct SQL
    <wdpa_pid>, <fc/fl>, <list of fields>"""
    where_name = (isinstance(wdpa_pid, int) or isinstance(wdpa_pid, float)) and str(wdpa_pid) or ('\'' + str(wdpa_pid) + '\'')
    
    where_clause = "wdpa_pid = " + where_name

    with arcpy.da.SearchCursor(data, fields_list, where_clause=where_clause) as cur:
        for each in cur:
            # if found return the first value
            return each
        
    # if not found return None
    return None

Run a few spot checks to see if the `get_row_info` function works

In [128]:
with arcpy.da.SearchCursor(old_data, ['wdpaid', 'name', 'desig']) as cur:
    for i, row in enumerate(cur):
        print(i, row[0], row[1], row[2])

0 4342.0 Verlandungszone am Ostende des Packer Stausees Naturschutzgebiet
1 169099.0 Hohe Tauern Nationalpark
2 169489.0 Teile des Edlacher Moores Geschützter Landschaftsteil
3 555513711.0 Naturpark Rosalia-Koglberg Naturpark
4 555513732.0 südlich Neuberggipfel Geschützte Biotope
5 555513733.0 Fasangarten Geschützte Biotope
6 555558910.0 Nähe Herrnholz-Stammersdorf Geschützte Biotope
7 4348.0 Inneres Pöllatal Naturschutzgebiet
8 169177.0 Pirkdorfer See Landschaftsschutzgebiet


In [129]:
# check if data is there
print(get_row_info('191', old_data, ['wdpaid', 'name']))

None


In [130]:
# check if data is there
print(get_row_info('4342', new_data, ['wdpaid', 'name']))

(4342.0, 'Verlandungszone am Ostende des Packer Stausees')


It can be annoying having to create a layer with a name that has been used before

Create a wrap for a function that does 1) checking if the layer name has been used, if used delete 2) create the layer

In [131]:
def create_layer(data, out_layer, *arg, **karg):
    if arcpy.Exists(out_layer):
        arcpy.Delete_management(out_layer)
    arcpy.MakeFeatureLayer_management(data, out_layer, *arg, **karg)

Create layers, which make things run a lot faster!

Input and output - dummy data from Austria

In [132]:
old_data_path = r".\data.gdb\old"
new_data_path = r".\data.gdb\new"

out_file = 'test_wdpa.csv'

# create layers
old_data = 'old'
new_data = 'new'

# create layers for two monthly releases
create_layer(old_data_path, old_data)
create_layer(new_data_path, new_data)

Get information about geometry

In [133]:
def geom_description(geom):
    if not isinstance(geom, arcpy.Geometry):
        raise Exception('arcpy Geometry object expected')
    
    try:
        return 'area:{0:.2f}km2; point_count:{1}; part_count:{2}; centroid(xy):{3:.6f}-{4:.6f}'.format(geom.getArea(units='SQUAREKILOMETERS'),
        geom.pointCount,
        geom.partCount,
        geom.centroid.X,
        geom.centroid.Y)

    except:
        return 'Error in retrieving geometry information'

The following section illustrates:
1. create a `deque` to faciliate first come first serve
2. start from the list of ids from the old data
3. iterate through the old data and compare to the new data to find records of DELETION (if no id found in new data) and CHANGE (iterate through the field list and compare field value)
4. iterate through the new data and compare to the old data to find records of ADDITION (simple `np.setdiff1d`)

In [134]:
# %%timeit about: 4min 13s for WDPA
from collections import deque
q = deque()

BASE = ['wdpa_pid', 'name', 'parent_iso3']
header = BASE + ['field', 'old_value', 'new_value']
fields_to_check = BASE + ['name', 'orig_name', 'desig', 'desig_eng', 'desig_type', 'iucn_cat', 'no_take', 'status', 'status_yr', 'SHAPE@']

FIELDS_LEN = len(fields_to_check)

o_idlist = list()

# header
q.append(header)

# start from old: DELETION and CHANGE
with arcpy.da.SearchCursor(old_data, fields_to_check) as cur:
    # for each old row, find and compare new row
    for o_row in cur:
        wdpa_pid = o_row[0]
        name = o_row[1]
        iso3 = o_row[2]
        n_row = get_row_info(wdpa_pid, new_data, fields_list=fields_to_check)
        
        # if no new row is found
        if not n_row:
            row = [wdpa_pid, name, iso3, 'DELETED', '', '']
            q.append(row)
        
        # if corresponding row found
        else:
            o_idlist.append(wdpa_pid)
            
            # Non spatial
            for i in range(FIELDS_LEN)[3:-1]:
                if n_row[i] != o_row[i]:
                    # row: pid, name, iso3, field, old_val, new_val
                    row = [wdpa_pid, name, iso3, fields_to_check[i], o_row[i], n_row[i]]
                    q.append(row)
            
            # spatial
            if n_row[-1] != o_row[-1]:
                row = [wdpa_pid, name, iso3, 'Geometry', geom_description(o_row[-1]), geom_description(n_row[-1])]
                q.append(row)

In [135]:
# start from new: ADD
n_idlist = list()
with arcpy.da.SearchCursor(new_data, BASE) as cur:
    for row in cur:
        n_idlist.append(row[0])

In [136]:
len(n_idlist), len(o_idlist)

(7, 7)

In [137]:
# find addition
import numpy as np
n_id = np.array(n_idlist)
o_id = np.array(o_idlist)

np.setdiff1d(o_id, n_id)

a_id = list(np.setdiff1d(n_id, o_id))

for wdpa_pid in a_id:
    n_row = get_row_info(wdpa_pid, new_data, BASE)

    wdpa_pid = n_row[0]
    name = n_row[1]
    iso3 = n_row[2]
    
    row = [wdpa_pid, name, iso3, 'ADDED', '', '']
    q.append(row)

In [138]:
# write outputs
import codecs
import copy

# shallow copy
qq = copy.deepcopy(q)

with codecs.open(out_file, mode='w', encoding='utf-8') as f:
    while qq:
        row = qq.popleft()
        # convert int and float to str, noting unicode
        # and-or trick: C equivalent boolean? x, y
        f.write(','.join(map(lambda x: (isinstance(x, int) or isinstance(x, float)) and str(x) or "\"" + x + "\"", row)) + '\n')

Take a look at the output file

In [139]:
result = pd.read_csv(out_file)

In [140]:
result

,wdpa_pid,name,parent_iso3,field,old_value,new_value
0,169099,Hohe Tauern,AUT,Geometry,area:805.59km2; point_count:6734; part_count:4...,Error in retrieving geometry information
1,169489,Teile des Edlacher Moores,AUT,desig,Geschützter Landschaftsteil,Naturschutzgebiet
2,169489,Teile des Edlacher Moores,AUT,desig_eng,Protected Landscape Section,Nature Reserve
3,169489,Teile des Edlacher Moores,AUT,Geometry,area:0.18km2; point_count:80; part_count:1; ce...,area:0.18km2; point_count:80; part_count:1; ce...
4,555513711,Naturpark Rosalia-Koglberg,AUT,Geometry,area:505.86km2; point_count:3818; part_count:1...,area:577.98km2; point_count:12095; part_count:...
5,555513732,südlich Neuberggipfel,AUT,desig_eng,Protected Biotope,protected biotopes
6,555513733,Fasangarten,AUT,desig_eng,Protected Biotope,protected biotopes
7,555558910,Nähe Herrnholz-Stammersdorf,AUT,desig_eng,Protected Biotope,protected biotopes
8,4348,Inneres Pöllatal,AUT,DELETED,NaN,NaN
9,169177,Pirkdorfer See,AUT,DELETED,NaN,NaN
